# Replaying experiments

In this tutorial, we will demonstrate how to reconstruct and visualize previous experiments from stored data. 

Let's import the relevant classes :

In [1]:
%load_ext param.ipython
import panel as pn

import larvaworld
from larvaworld.lib import reg, util

# Import the simulation class
from larvaworld.lib.sim import ReplayRun

# Import the configuration classes
from larvaworld.lib.reg.generators import ReplayConf, ReplayConfUnit, ReplayConfGroup

larvaworld.VERBOSE = 1

Welcome to the param IPython extension! (https://param.holoviz.org/)
Available magics: %params
Initializing larvaworld registry
Registry configured!


The configuration parameters passed to the simulation class are managed by three classes :

- ReplayConfGroup : group-level parameters
- ReplayConfUnit : agent-level parameters
- ReplayConf : all parameters (including the above)

In [2]:
# Show the attributes of the ReplayConfGroup class
%params ReplayConfGroup

# Show the attributes of the ReplayConfGroup class as a nested dictionary
ReplayConfGroup.param

Name,Default,Type,Range
agent_ids,[],List,"(0, None)"
env_params,None,OptionalSelector,"nullable '4corners', 'CS_UCS_off_food', 'CS_UCS_on_food', 'CS_UCS_on_food_x2', 'arena_1000mm', 'arena_200mm', 'arena_500mm', 'arena_50mm_diffusion', 'dish', 'dish_40mm', 'double_patch', 'focus', 'food_at_bottom', 'food_grid', 'game', 'maze', 'mid_odor_diffusion', 'mid_odor_gaussian', 'multi_patch', 'odor_gaussian_square', 'odor_gradient', 'patch_grid', 'patchy_food', 'puff_arena_bordered', 'random_food', 'single_odor_patch', 'single_patch', 'single_puff', 'thermo_arena', 'uniform_food', 'windy_arena', 'windy_arena_bordered', 'windy_blob_arena'"
name,'ReplayConfGroup',String,nullable constant
track_point,-1,Integer,
transposition,None,OptionalSelector,"nullable 'origin', 'arena', 'center'"


Parameters of 'ReplayConfGroup'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name           Value        Type          Bounds     Mode  

agent_ids        []         List        (0, None)    V RW  
transposition   None  OptionalSelector             V RW AN 
track_point      -1       Integer        (-1, 12)    V RW  
env_params      None  OptionalSelector             V RW AN 

Parameter docstrings:

agent_ids:     Whether to only display some larvae of the dataset, defined by their indexes.
transposition: Whether to transpose the dataset spatial coordinates.
track_point:   The midline point to use for defining the larva position.
env_params:    Selection among stored Env configurations by ID

In [3]:
# Show the attributes of the ReplayConfUnit class
%params ReplayConfUnit

# Show the attributes of the ReplayConfUnit class as a nested dictionary
ReplayConfUnit.param

Name,Default,Type,Range
close_view,False,Boolean,
fix_point,None,OptionalPositiveInteger,nullable >=0
fix_segment,None,OptionalSelector,"nullable 'rear', 'front'"
name,'ReplayConfUnit',String,nullable constant


Parameters of 'ReplayConfUnit'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name          Value            Type            Bounds    Mode  

close_view    False          Boolean                     V RW  
fix_segment    None      OptionalSelector              V RW AN 
fix_point      None  OptionalPositiveInteger  (0, 12)  V RW AN 

Parameter docstrings:

close_view:  Whether to visualize a small arena on close range.
fix_segment: Whether to additionally fixate the above or below body segment.
fix_point:   Whether to fixate a specific midline point to the center of the screen. Relevant when replaying a single larva track.

In [4]:
# Show the attributes of the ReplayConf class
%params ReplayConf

# Show the attributes of the ReplayConf class as a nested dictionary
ReplayConf.param

Name,Default,Type,Range
agent_ids,[],List,"(0, None)"
close_view,False,Boolean,
draw_Nsegs,None,OptionalPositiveInteger,nullable >=0
env_params,None,OptionalSelector,"nullable '4corners', 'CS_UCS_off_food', 'CS_UCS_on_food', 'CS_UCS_on_food_x2', 'arena_1000mm', 'arena_200mm', 'arena_500mm', 'arena_50mm_diffusion', 'dish', 'dish_40mm', 'double_patch', 'focus', 'food_at_bottom', 'food_grid', 'game', 'maze', 'mid_odor_diffusion', 'mid_odor_gaussian', 'multi_patch', 'odor_gaussian_square', 'odor_gradient', 'patch_grid', 'patchy_food', 'puff_arena_bordered', 'random_food', 'single_odor_patch', 'single_patch', 'single_puff', 'thermo_arena', 'uniform_food', 'windy_arena', 'windy_arena_bordered', 'windy_blob_arena'"
fix_point,None,OptionalPositiveInteger,nullable >=0
fix_segment,None,OptionalSelector,"nullable 'rear', 'front'"
name,'ReplayConf',String,nullable constant
overlap_mode,False,Boolean,
refDir,None,String,nullable
refID,None,OptionalSelector,nullable 'exploration.30controls'


Parameters of 'ReplayConf'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name            Value            Type              Bounds      Mode  

close_view      False          Boolean                         V RW  
fix_segment      None      OptionalSelector                  V RW AN 
fix_point        None  OptionalPositiveInteger    (0, 12)    V RW AN 
agent_ids         []             List            (0, None)     V RW  
transposition    None      OptionalSelector                  V RW AN 
track_point       -1           Integer            (-1, 12)     V RW  
env_params       None      OptionalSelector                  V RW AN 
refID            None      OptionalSelector                  V RW AN 
refDir           None           String                       V RW AN 
time_range       None   OptionalPositiveRange   (0.0, None)  V RW AN 
overlap_mode    False          B

Now we will specify the dataset to be reconstructed by its unique ID.

It is also possible to locate it by the directory where it is stored 

In [ ]:
refID = reg.default_refID
# refID = 'Rehydration/AttP2.Deprived'
# refID = 'exploration.dish'
# refID = reg.default_refID
# refID = 'naive_locomotion.20controls'
# refID = 'exploration.150controls'
d = reg.loadRef(refID)

We will specify a number of configuration sets as dictionaries :

In [ ]:
replay_confs = [
    ("normal", {"time_range": (40, 60)}),
    ("dispersal", {"transposition": "origin", "time_range": (0, 20)}),
    (
        "fixed_point",
        {
            "agent_ids": [0],
            "close_view": True,
            "fix_point": 6,
            "time_range": (80, 100),
        },
    ),
    (
        "fixed_segment",
        {
            "agent_ids": [0],
            "close_view": True,
            "fix_point": 6,
            "fix_segment": "rear",
            "time_range": (100, 130),
        },
    ),
    (
        "fixed_overlap",
        {
            "agent_ids": [0],
            "close_view": True,
            "fix_point": 6,
            "fix_segment": "front",
            "overlap_mode": True,
        },
    ),
    ("2segs", {"draw_Nsegs": 2, "time_range": (80, 100)}),
    ("all_segs", {"draw_Nsegs": 11, "time_range": (80, 100)}),
]

In [ ]:
# A method that runs the replay simulation
def run_replay(mode):
    rep = ReplayRun(
        parameters=ReplayConf(refID=refID, **replay_confs[mode]).nestedConf,
        id=f"{refID}_replay_{mode}",
        dir=f"./media/{mode}",
        screen_kws={"vis_mode": "video", "show_display": True},
    )
    _ = rep.run()

In [ ]:
# Run a normal replay of the dataset
run_replay("normal")

In [ ]:
# Run a reconstructed dispersal experiment where trajectories have benn transposed to the origin
run_replay("dispersal")

In [ ]:
# Substitute the larva body contour by a bisegmental body
run_replay("2segs")

In [ ]:
# ... or by a body with all segments, making use of all the midline points available
run_replay("all_segs")

In [ ]:
# Now let's examine a single individual. Fixate a midline point of the larva body to the arena center
run_replay("fixed_point")

In [ ]:
# Now fixate a midline segment along the y axis
run_replay("fixed_segment")

In [ ]:
# And collapse the entire video to a single image to visualize the flexibility of each segment
run_replay("fixed_overlap")